In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontManager


## Loading Machine Learning libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer


from mpl_toolkits.mplot3d import Axes3D

pd.set_option("display.max_rows", 8)
pd.options.mode.chained_assignment = None

%matplotlib inline
%config InlineBacked.figure_format = "retina"

In [4]:
stopwords = pd.read_csv("../data/stopwords.txt", 
                       header = None, 
                       names = ['Stopwords'])

df = pd.read_csv("../data/corp_text.csv")
df.head(5)

,Unnamed: 0,shixin_id,duty
0,1,2905,"被告上海纺友贸易有限公司应于本判决生效后十日内支付原告江苏盛虹面料有限公司货款人民币1,13..."
1,6,3650,"一、被告上海栩瑞电气设备有限公司确认尚欠原告上海宝临超亚电器有限公司货款302,725元，原..."
2,16,11617,原告上海玖峰数码科技有限公司于本判决生效之日起十日内支付被告张宁顺违法解除劳动合同赔偿金93...
3,21,15642,被执行人于2013年4月25前向申请执行人支付工资人民币5万元；被执行人于2013年5月15...
4,22,16229,一、原告上海顶天钢铁发展有限公司与被告内蒙古兴托物资有限公司于2007年1月11日签订的《内...


In [5]:
import jieba ## Chinese Langauge Segementation tool;

In [6]:
list(jieba.cut(df.duty[3], cut_all=True))  ## test for iname;

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ys/98s0q77d0bl4mpqsp4lf7c740000gn/T/jieba.cache
Loading model cost 0.768 seconds.
Prefix dict has been built successfully.


['被执行人',
 '执行',
 '行人',
 '于',
 '2013',
 '年',
 '4',
 '月',
 '25',
 '前',
 '向',
 '申请',
 '执行',
 '行人',
 '支付',
 '工资',
 '人民',
 '人民币',
 '5',
 '万元',
 '；',
 '被执行人',
 '执行',
 '行人',
 '于',
 '2013',
 '年',
 '5',
 '月',
 '15',
 '前',
 '向',
 '申请',
 '执行',
 '行人',
 '支付',
 '工资',
 '人民',
 '人民币',
 '5',
 '万元',
 '；',
 '上述',
 '款项',
 '支付',
 '至',
 '申请',
 '执行',
 '行人',
 '原',
 '工资',
 '工资卡',
 '账户',
 '（',
 '汇丰',
 '汇丰银行',
 '银行',
 '）；',
 '如',
 '被执行人',
 '执行',
 '行人',
 '于',
 '2013',
 '年',
 '4',
 '月',
 '30',
 '日',
 '之前',
 '未',
 '支付',
 '第一',
 '第一笔',
 '一笔',
 '款项',
 '，',
 '申请',
 '执行',
 '行人',
 '可以',
 '就',
 '所有',
 '剩余',
 '余款',
 '款项',
 '一并',
 '申请',
 '强制',
 '强制执行',
 '执行',
 '；',
 '申请',
 '执行',
 '行人',
 '放弃',
 '其他',
 '仲裁',
 '请求',
 '。']

In [7]:
row,col = df.shape

df['cutword'] = 'cutword'
for i in np.arange(row):
    cutwords = list(jieba.cut(df.duty[i], cut_all=True))
    cutwords = pd.Series(cutwords)[pd.Series(cutwords).apply(len)>1]
    df.cutword[i] = cutwords.values

In [37]:
df.cutword[1:5]

1    [被告, 上海, 电气, 电气设备, 设备, 备有, 有限, 有限公司, 公司, 确认, 原...
2    [原告, 上海, 数码, 数码科技, 科技, 有限, 有限公司, 公司, 判决, 生效, 之...
3    [被执行人, 执行, 行人, 月前, 申请, 执行, 行人, 支付, 工资, 人民, 人民币...
4    [原告, 上海, 顶天, 钢铁, 发展, 有限, 有限公司, 公司, 被告, 内蒙, 内蒙古...
Name: cutword, dtype: object

In [8]:
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

/usr/local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [9]:
dictionary = Dictionary(df.cutword)
corpus = [dictionary.doc2bow(word) for word in df.cutword]

In [10]:
lda = LdaModel(corpus=corpus, id2word = dictionary, num_topics = 100, random_state=12)

In [16]:
lda.print_topics(100)

[(0,
  '0.163*"服务" + 0.042*"系统" + 0.039*"公司" + 0.037*"顺位" + 0.036*"智能" + 0.035*"有限公司" + 0.033*"有限" + 0.032*"上海" + 0.022*"健身" + 0.021*"贸发"'),
 (1,
  '0.083*"上海" + 0.065*"公司" + 0.064*"有限公司" + 0.063*"有限" + 0.052*"原告" + 0.044*"被告" + 0.022*"反诉" + 0.020*"之日" + 0.020*"房屋" + 0.018*"返还"'),
 (2,
  '0.088*"2018" + 0.049*"000" + 0.047*"借款" + 0.045*"本金" + 0.034*"利息" + 0.031*"计算" + 0.031*"清偿" + 0.030*"上海" + 0.029*"年利率" + 0.029*"实际"'),
 (3,
  '0.291*"电气" + 0.133*"37" + 0.123*"土地" + 0.093*"占用" + 0.054*"950" + 0.042*"海航" + 0.021*"地上" + 0.016*"45" + 0.012*"设施" + 0.011*"诉讼"'),
 (4,
  '0.064*"有限" + 0.056*"公司" + 0.056*"有限公司" + 0.044*"能源" + 0.043*"体育" + 0.038*"被告" + 0.034*"河南" + 0.024*"电工" + 0.018*"四川" + 0.017*"原告"'),
 (5,
  '0.215*"物流" + 0.048*"平安" + 0.045*"分公司" + 0.045*"公司" + 0.027*"被告" + 0.025*"有限" + 0.024*"原告" + 0.023*"有限公司" + 0.021*"第三" + 0.018*"判决"'),
 (6,
  '0.194*"68" + 0.165*"56" + 0.091*"货物" + 0.071*"860" + 0.047*"不能" + 0.037*"430" + 0.032*"745" + 0.023*"交付" + 0.020*"806" + 0.016*"规格"'),
 (7,
  '0

In [14]:
import pyLDAvis
import pyLDAvis.gensim 

In [15]:

# feed the LDA model into the pyLDAvis instance
lda_viz = pyLDAvis.gensim.prepare(lda , corpus=corpus, dictionary=dictionary)

/usr/local/lib/python3.7/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
